In [1]:
import os
import json
import numpy as np
from subgraph_utils import *
import pandas as pd
from utils import *
from utils import GraphAlgos

/apps/conda/isha/envs/quacer-b/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/apps/conda/isha/envs/quacer-b/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/apps/conda/isha/envs/quacer-b/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/apps/conda/isha/envs/quacer-b/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
prime_folder = '/share/quacer_c/prime'
actual_rels = json.load(open(os.path.join(prime_folder, 'actual_rels.json')))
edge2src = json.load(open(os.path.join(prime_folder, 'edge2src.json')))
graph = json.load(open(os.path.join(prime_folder, 'graph.json')))
id2name = json.load(open(os.path.join(prime_folder, 'id2name.json')))
id2source = json.load(open(os.path.join(prime_folder, 'id2source.json')))
rels = json.load(open(os.path.join(prime_folder, 'rels.json')))
graph_text_edge = json.load(open(os.path.join(prime_folder, 'graph_text_edge.json')))

In [3]:
entity_aliases = {}
relation_aliases = {}
for k, v in id2name.items():
    if k.startswith('RIDR'):
        relation_aliases[k] = [v]
    else:
        entity_aliases[k] = [v]

In [4]:
graph_algos = GraphAlgos(graph, entity_aliases, relation_aliases)

In [5]:
from discovery_functions import least_side_effects_discoverer

discovery_funcs = [least_side_effects_discoverer]

query_generator = CustomQueryGenerator(graph_algos, discovery_funcs)

In [6]:
query_results = query_generator.get_single_query(0, id2name=id2name)
query_data = query_generator.generate_query_data(query_results, id2name=id2name, 
                                                 graph_text_edge=graph_text_edge,
                                                 tokenizer=None)

In [7]:
query_data

{'query': "Which drug used to treat Kaposi's sarcoma (disease) has the least number of side effects?",
 'correct_answers': ['Vinblastine'],
 'path_id': ['ENID22304',
  'ENID84469',
  'ENID33730',
  'ENID84361',
  'ENID93625',
  'ENID85214',
  'ENID23000',
  'ENID25235',
  'ENID84855',
  'ENID22934',
  'ENID23002',
  'ENID85849',
  'ENID33737',
  'ENID23073',
  'ENID15107',
  'ENID22982',
  'ENID84409',
  'ENID25201',
  'ENID23139',
  'ENID25569',
  'ENID23157',
  'ENID22457',
  'ENID25644',
  'ENID92920',
  'ENID22458',
  'ENID23158',
  'ENID23799',
  'ENID22938',
  'ENID32589'],
 'path_en': ['Hearing impairment',
  'Photophobia',
  'Leukopenia',
  'Angina pectoris',
  'Mandibular pain',
  'Bone pain',
  'Seizure',
  'Ileus',
  'Polyneuropathy',
  'Thrombocytopenia',
  'Headache',
  'Poor appetite',
  'Peptic ulcer',
  'Alopecia',
  'Vinblastine',
  'Pain',
  'Nausea',
  'Vertigo',
  'Sensory impairment',
  'Granulocytopenia',
  'Vomiting',
  'Peripheral neuropathy',
  'Paresthesia',
 

In [8]:
relation_aliases

{'RIDR0': ['ppi_gene/protein'],
 'RIDR1': ['carrier_gene/protein'],
 'RIDR2': ['enzyme_gene/protein'],
 'RIDR3': ['target_gene/protein'],
 'RIDR4': ['transporter_gene/protein'],
 'RIDR5': ['contraindication_disease'],
 'RIDR6': ['indication_disease'],
 'RIDR7': ['off-label use_disease'],
 'RIDR8': ['synergistic interaction_drug'],
 'RIDR9': ['associated with_effect/phenotype'],
 'RIDR10': ['parent-child_effect/phenotype'],
 'RIDR11': ['phenotype absent_effect/phenotype'],
 'RIDR12': ['phenotype present_effect/phenotype'],
 'RIDR13': ['phenotype present_disease'],
 'RIDR14': ['associated with_disease'],
 'RIDR15': ['parent-child_disease'],
 'RIDR16': ['side effect_effect/phenotype'],
 'RIDR17': ['parent-child_biological_process'],
 'RIDR18': ['parent-child_molecular_function'],
 'RIDR19': ['parent-child_cellular_component'],
 'RIDR20': ['interacts with_molecular_function'],
 'RIDR21': ['interacts with_cellular_component'],
 'RIDR22': ['interacts with_biological_process'],
 'RIDR23': ['i

In [10]:
len(list(graph.keys()))

129375

In [20]:
# Random disease sampling
disease_nodes = list(graph.keys())
random.shuffle(disease_nodes)
unique_diseases = set()
for disease in disease_nodes:
    disease_data = graph[disease]
    drugs = [node for node, rel in disease_data.items() if rel == 'RIDR36']
    
    if len(drugs) > 1:
        drug_side_effects = {}
        essential_edges = []
        context_edges = []
        
        for drug in drugs:
            edge = (disease, drug)
            context_edges.append(edge)
        
        for drug in drugs:
            side_effects = [node for node, rel in graph[drug].items() 
                            if rel == 'RIDR16']
            drug_side_effects[drug] = side_effects
            
            for se in side_effects:
                edge = (drug, se)
                context_edges.append(edge)
            
        min_count = min(len(se) for se in drug_side_effects.values())
        min_drugs = [d for d, se in drug_side_effects.items() 
                    if len(se) == min_count]
        unique_diseases.add(disease)
len(unique_diseases), len(unique_diseases)*6

(925, 5550)

In [11]:
disease_rels = []
for rel, rel_name in relation_aliases.items():
    if '_disease' in rel_name[0]:
        disease_rels.append(rel)
disease_rels

['RIDR5', 'RIDR6', 'RIDR7', 'RIDR13', 'RIDR14', 'RIDR15', 'RIDR24', 'RIDR39']

In [16]:
relation_aliases

{'RIDR0': ['ppi_gene/protein'],
 'RIDR1': ['carrier_gene/protein'],
 'RIDR2': ['enzyme_gene/protein'],
 'RIDR3': ['target_gene/protein'],
 'RIDR4': ['transporter_gene/protein'],
 'RIDR5': ['contraindication_disease'],
 'RIDR6': ['indication_disease'],
 'RIDR7': ['off-label use_disease'],
 'RIDR8': ['synergistic interaction_drug'],
 'RIDR9': ['associated with_effect/phenotype'],
 'RIDR10': ['parent-child_effect/phenotype'],
 'RIDR11': ['phenotype absent_effect/phenotype'],
 'RIDR12': ['phenotype present_effect/phenotype'],
 'RIDR13': ['phenotype present_disease'],
 'RIDR14': ['associated with_disease'],
 'RIDR15': ['parent-child_disease'],
 'RIDR16': ['side effect_effect/phenotype'],
 'RIDR17': ['parent-child_biological_process'],
 'RIDR18': ['parent-child_molecular_function'],
 'RIDR19': ['parent-child_cellular_component'],
 'RIDR20': ['interacts with_molecular_function'],
 'RIDR21': ['interacts with_cellular_component'],
 'RIDR22': ['interacts with_biological_process'],
 'RIDR23': ['i

In [12]:
disease_nodes = set()
for node in graph.keys():
    for node2, rel in graph[node].items():
        if rel in disease_rels:
            disease_nodes.add(node2)
len(disease_nodes)

17080

In [18]:
disease_w_drug = set()
no_disease_drug = set()
for disease_node in disease_nodes:
    for node, rel in graph[disease_node].items():
        if rel == 'RIDR36' or rel =='RIDR37':
            disease_w_drug.add(disease_node)
            break

for disease_node in disease_nodes:
    if disease_node not in disease_w_drug:
        no_disease_drug.add(disease_node)
len(disease_w_drug), len(no_disease_drug)

(1548, 15532)

In [25]:
list(no_disease_drug)[1]

'ENID27773'

In [17]:
for node1, node_data in graph.items():
    for node2, rel in node_data.items():
        if rel == 'RIDR24':
            print(node1, node2, id2name[node1], id2name[node2])
            break

ENID61678 ENID36414 1-hydroxypyrene male infertility
ENID61679 ENID36414 1-naphthol male infertility
ENID61680 ENID35831 2,2',3',4,4',5-hexachlorobiphenyl respiratory tract infectious disease
ENID61681 ENID36242 2,2',3,5,5',6-hexachlorobiphenyl fatty liver disease
ENID61682 ENID35866 2,2',4,4',5-brominated diphenyl ether precursor lymphoblastic lymphoma/leukemia
ENID61683 ENID38512 2,2',4,4',5-pentachlorobiphenyl female infertility
ENID61684 ENID35866 2,2',4,4'-tetrabromodiphenyl ether precursor lymphoblastic lymphoma/leukemia
ENID61685 ENID29614 2,3,3',4,4'-pentachlorobiphenyl osteoporosis
ENID61686 ENID29614 2,3',4,4',5-pentachlorobiphenyl osteoporosis
ENID61687 ENID33575 2,4,4',5-tetrachlorobiphenyl diabetes mellitus (disease)
ENID61688 ENID35831 2,4,4'-trichlorobiphenyl respiratory tract infectious disease
ENID61689 ENID35831 2,4,5,2',5'-pentachlorobiphenyl respiratory tract infectious disease
ENID61690 ENID35831 2,5,2',5'-tetrachlorobiphenyl respiratory tract infectious disease
EN

In [26]:
entity_aliases['ENID27773']

['kyphoscoliosis-lateral tongue atrophy-hereditary spastic paraplegia syndrome']

In [27]:
for node2, rel in graph['ENID27773'].items():
    print(node2, id2name[node2], rel, id2name[rel])

ENID24252 Cerebellar atrophy RIDR11 phenotype absent_effect/phenotype
ENID23017 Cerebral atrophy RIDR11 phenotype absent_effect/phenotype
ENID23411 Abnormal circulating creatine kinase concentration RIDR11 phenotype absent_effect/phenotype
ENID22877 Talipes equinovarus RIDR12 phenotype present_effect/phenotype
ENID22678 Lower limb spasticity RIDR12 phenotype present_effect/phenotype
ENID25842 Delayed gross motor development RIDR12 phenotype present_effect/phenotype
ENID24294 Lower limb hyperreflexia RIDR12 phenotype present_effect/phenotype
ENID23471 Kyphoscoliosis RIDR12 phenotype present_effect/phenotype
ENID23483 Muscle spasm RIDR12 phenotype present_effect/phenotype
ENID85599 Difficulty standing RIDR12 phenotype present_effect/phenotype
ENID84403 Progressive spastic paraplegia RIDR12 phenotype present_effect/phenotype
ENID24614 Lower limb amyotrophy RIDR12 phenotype present_effect/phenotype
ENID26137 Proximal muscle weakness in upper limbs RIDR12 phenotype present_effect/phenotype


In [29]:
relation_aliases

{'RIDR0': ['ppi_gene/protein'],
 'RIDR1': ['carrier_gene/protein'],
 'RIDR2': ['enzyme_gene/protein'],
 'RIDR3': ['target_gene/protein'],
 'RIDR4': ['transporter_gene/protein'],
 'RIDR5': ['contraindication_disease'],
 'RIDR6': ['indication_disease'],
 'RIDR7': ['off-label use_disease'],
 'RIDR8': ['synergistic interaction_drug'],
 'RIDR9': ['associated with_effect/phenotype'],
 'RIDR10': ['parent-child_effect/phenotype'],
 'RIDR11': ['phenotype absent_effect/phenotype'],
 'RIDR12': ['phenotype present_effect/phenotype'],
 'RIDR13': ['phenotype present_disease'],
 'RIDR14': ['associated with_disease'],
 'RIDR15': ['parent-child_disease'],
 'RIDR16': ['side effect_effect/phenotype'],
 'RIDR17': ['parent-child_biological_process'],
 'RIDR18': ['parent-child_molecular_function'],
 'RIDR19': ['parent-child_cellular_component'],
 'RIDR20': ['interacts with_molecular_function'],
 'RIDR21': ['interacts with_cellular_component'],
 'RIDR22': ['interacts with_biological_process'],
 'RIDR23': ['i

In [30]:
disease_phenotypes = {}
for node in disease_nodes:
    disease_phenotypes[node] = []
    for node2, rel in graph[node].items():
        if rel == 'RIDR12':
            disease_phenotypes[node].append(node2)

phenotypes2disease = {}
for disease, phenotypes in disease_phenotypes.items():
    phenotype_tuple = tuple(phenotypes)
    if phenotype_tuple not in phenotypes2disease:
        phenotypes2disease[phenotype_tuple] = []
    phenotypes2disease[phenotype_tuple].append(disease)

pehnotype_goods = []
for phenotypes, diseases in phenotypes2disease.items():
    if len(diseases) == 1:
        pehnotype_goods.append((phenotypes, diseases[0]))
len(pehnotype_goods)

6540

In [36]:
ph_id2len = {}
for phenotype_tup in pehnotype_goods:
    names_phenotypes = [id2name[phenotype] for phenotype in phenotype_tup[0]]
    disease = phenotype_tup[1]
    # print(names_phenotypes, id2name[disease])
    # break
    ph_id2len[phenotype_tup] = len(names_phenotypes)

len2num_phe = {}
for ph_id, length in ph_id2len.items():
    if length not in len2num_phe:
        len2num_phe[length] = 0
    len2num_phe[length] += 1

max_le = 30
good_phe_ids = []
for i, pheno_tup in enumerate(pehnotype_goods):
    if ph_id2len[pheno_tup] <= max_le:
        good_phe_ids.append(pheno_tup)
len(good_phe_ids)

4958

In [39]:
for phe_tup in good_phe_ids:
    names_phenotypes = [id2name[phenotype] for phenotype in phe_tup[0]]
    print(names_phenotypes, id2name[phe_tup[1]])
    break


['Delayed speech and language development', 'Developmental regression', 'Seizure', 'X-linked recessive inheritance', 'Intellectual disability'] epsilon-trimethyllysine hydroxylase deficiency
